<h1><center>Webhook for Webex Teams</center></h1>

#### These are the following API Calls Supported by Webex Team

> **List Webhooks**        (GET)<br>
> **Create a Webhook**     (POST)<br>
> **Get Webhook Details**  (GET)<br>
> **Update a Webhook**     (PUT)<br>
> **Delete a Webhook**     (DELETE)

In [ ]:
# These libraries are required to run the above stated libraries
import requests
import json

`Below details contains BOT Access Token and Header with expected request and response in JSON`

In [ ]:
token="<token-id>" 
# Mention the token ID achieved when you have created the BOT
# Reference: https://developer.webex.com/my-apps/new/bot

#This is a global header declared that will be used by all the API calls
headers={'content-type': "application/json",
         'authorization':'Bearer {}'.format(token),
         'accept':"application/json"}

#### --------------------------------------------------------------------------------- RoomID for a Group ---------------------------------------------------------------------------------

`This Function is not actually a part of Webhook Section
But will help us to retain the RoomID for the Group Name Mentioned if BOT present in that group`

In [ ]:
def grp_check(grp_name):
    b_token = "Bearer "+token
    group_check="https://webexapis.com/v1/rooms?type=group"
    headers={'content-type': "application/json",
             'authorization':b_token,
             'accept':"application/json"}
    response = requests.request("GET", group_check, headers=headers)
    data=response.json()
    groups = data["items"]
#     print(groups)
#     print("\n")
    for group in groups:
#         print(group)
#         print("\n")
        if group['title'] == grp_name:
            result = group['id']
            status=True
            break
        else:
            result="The mentioned Group is not associated with the BOT. Please make sure that the added spelling is correct with your group name or Please add the BOT into the group if missing"
            status=False
    return status,result

grp_check('Group Name') #Mention Webex Group Name

#### --------------------------------------------------------------------------------- List All Webhooks ---------------------------------------------------------------------------------

`
Below Function will list all the Webhook associated with the BOT Access Token
Response Payload is in JSON Format includes Name, targetUrl, Resources, Event, Filter, OrgID, createdBy, appId, ownedBy, status, created`

In [ ]:
def list_webhook():
    url="https://webexapis.com/v1/webhooks?max=100"
    response=requests.request("GET", url, headers=headers)
    data=response.json()['items']
    return data

list_webhook()

#### --------------------------------------------------------------------------------- Create a new Webhook ---------------------------------------------------------------------------------

`
Below Function will create a new Webhook
Request Payload is in JSON Format requires Name, targetUrl, Resources and Event
These are some optional fields Filter and Secret`

In [ ]:
def create_webhook(roomID,webhook_name,webhook_url):
    url="https://webexapis.com/v1/webhooks"
    payload=json.dumps({
        "name": "{}".format(webhook_name),
        "targetUrl": "{}".format(webhook_url),
        "resource": "messages",
        "event": "created",
        "filter": "roomId={}".format(roomID) # Here, mentioned that BOT is accessible via associated Group only.
    })
    response = requests.request("POST", url, data=payload, headers=headers)
    print("Here's the Webhook Details !!")
    if response.status_code==200:
        print(response.json())
    else:
        print('Error Detected !!')
        print(response.json())
        

In [ ]:
status,roomID=grp_check('Group Name') # Mention the Group Name
webhook_name='Testing Webhook' # Mention the Name for the Webhook
webhook_url='https://test.ngrok.io' # Mention the Webhook URL
create_webhook(roomID,webhook_name,webhook_url)

#### --------------------------------------------------------------------------------- Webhook Details---------------------------------------------------------------------------------

<b>For Webhook details you need Webhook ID:</b><br><br>
`
{
  "id": "Y2lzY29zcGFyazovL3VzL1dFQkhPT0svOTZhYmMyYWEtM2RjYy0xMWU1LWExNTItZmUzNDgxOWNkYzlh",
  "name": "My Awesome Webhook",
  "targetUrl": "https://example.com/mywebhook",
  "resource": "messages",
  "event": "created",
  "filter": "roomId=Y2lzY29zcGFyazovL3VzL1JPT00vYmJjZWIxYWQtNDNmMS0zYjU4LTkxNDctZjE0YmIwYzRkMTU0",
  "secret": "86dacc007724d8ea666f88fc77d918dad9537a15",
  "status": "active",
  "created": "2015-10-18T14:26:16+00:00"
}`


`
Below Function will get the details of particular Webhook, Using Webhook ID
Response Payload is in JSON Format includes ID, Name, targetUrl, Resources, Event, Filter, Secret, Status, Created`

In [ ]:
def get_webhook_details(id):
    url="https://webexapis.com/v1/webhooks/{}".format(id)
    response = requests.request("GET", url, headers=headers)
    if response.status_code==200:
        print(response.json())
    else:
        print('Error Detected !!')
        print(response.json())

#Mention the Webhook ID, Recieved when Webhook has been created
get_webhook_details('Webhook ID')

#### --------------------------------------------------------------------------------- Update Webhook Details---------------------------------------------------------------------------------

<b>These Information can be updated via below API call:</b><br><br>
`
{
  "name": "My Awesome Webhook",
  "targetUrl": "https://example.com/mywebhook",
  "secret": "86dacc007724d8ea666f88fc77d918dad9537a15",
  "status": "active"
}`

**Note:** ***name and targetUrl are required***

`Below Function will update the particular Webhook, Using Webhook ID
Here Request Payload is in JSON Format as Name and targetUrl, Both of them are required`

In [ ]:
def update_webex_details(id):
    url="https://webexapis.com/v1/webhooks/{}".format(id)
    webhook_name="demo_change"
    webhook_url="https://test.ngrok.io"
    
    payload=json.dumps({
        "name": "{}".format(webhook_name),
        "targetUrl": "{}".format(webhook_url)
    })
    response = requests.request("PUT", url,data=payload, headers=headers)
    if response.status_code==200:
        print(response.json())
    else:
        print('Error Detected !!')
        print(response.json())
        
#The string contains the Webhook ID, Recieved when Webhook has been created
update_webex_details("Webhook ID")

#### --------------------------------------------------------------------------------- Delete a Webhook---------------------------------------------------------------------------------

`Below function will delete all the Webhooks created for this BOT, using List All Webhooks API`

In [ ]:
def delete_all_webhook():
    for web in list_webhook():
        url="https://webexapis.com/v1/webhooks/{}".format(web['id'])
        response = requests.request("DELETE", url, headers=headers)
        print('Successfully Deleted {} webhook'.format(web['name']))
delete_all_webhook()      

`Delete a particular Webhook based on the Webhook ID Provided`

In [ ]:
def delete_webhook(id):
    url="https://webexapis.com/v1/webhooks/{}".format(id)
    response = requests.request("DELETE", url, headers=headers)
    if response.status_code==204:
        print('Successfully Deleted !!')
    else:
        print('Error Detected !!')
        print(response.json())

#Mention the Webhook ID, Recieved when Webhook has been created
delete_webhook('Webhook ID')